In [ ]:
image_name = '{}sc{:06d}{}.png'.format(some_string, scene_id, some_other_string)
corresponding_extrinsic_matrix_name = '{}sc{:06d}{}_RT.txt'.format(some_string, scene_id, some_other_string)

### Organize dataset

In [3]:
import os
import shutil
import glob
import json
import numpy as np

dataset_root = '/disk1/sliangal/uORF/dataset'
os.makedirs(dataset_root, exist_ok=True)
ori_dataset_root = '/disk1/yichen/front3d_ngp/'

scene_list = ['3dfront_0110_00']

for scene_name in scene_list:
    dataset_dir = os.path.join(dataset_root, scene_name)
    if os.path.isdir(dataset_dir):
        shutil.rmtree(dataset_dir)
    os.makedirs(dataset_dir)

    splits = scene_name.split('_')
    scene_id = f'3dfront_sc{splits[1]}{splits[2]}'

    with open(os.path.join(ori_dataset_root, scene_name, 'train/transforms.json'), 'r') as f:
        xform_json = json.load(f)
    room_bbox = np.array(xform_json['room_bbox'])
    center_offset = (room_bbox[1] + room_bbox[0]) / 2

    frames = xform_json['frames']
    # image_list = sorted(glob.glob(os.path.join(ori_dataset_root, scene_name, 'train/images/*.jpg')))
    # assert len(frames)==len(image_list)

    for frame in frames:
        image_id = frame['file_path'].split('/')[-1].split('.')[0]
        frame_name = f'{scene_id}_{image_id}'
        shutil.copyfile(os.path.join(ori_dataset_root, scene_name, 'train', frame['file_path']),
                        os.path.join(dataset_dir, f'{frame_name}.jpg'))

        rt = np.array(frame['transform_matrix'])
        rt[:3, 3] -= center_offset
        np.savetxt(os.path.join(dataset_dir, f'{frame_name}_RT.txt'), rt)

